# Asset Condition Monitoring Component Assessment Map

**Category:** CSV Source
**Scope:** Asset Condition Monitoring Technology   
**Author:** Michael Moyer   
**Created:** 01-13-2026   
**Last Updated:** 01-13-2026  

**Purpose:** Primary and Secondary Correspondence

Notebook gathers the data, organizes, cleans and provides a brief report on it. 
        
**Use Case:** Use this notebook to ingest component coverages.

---

In [1]:
# Cell 1: Load and inspect
import pandas as pd

df_rules = pd.read_csv('component_map.csv')
print(df_rules.head())
print(f"\nColumns: {df_rules.columns.tolist()}")



                      Component Type IR LU VI UL MC ZD GM
0                   Electric Motors   S  -  P  S  P  -  -
1     Rolling Element Bearings (Oil)  -  P  P  S  -  -  -
2  Rolling Element Bearings (Grease)  -  -  P  P  -  -  -
3         Gearboxes / Speed Reducers  S  P  -  S  -  -  S
4                        Belt Drives  -  -  P  -  -  -  -

Columns: ['Component Type', 'IR', 'LU', 'VI', 'UL', 'MC', 'ZD', 'GM']


In [2]:
# Cell 2: Clean up the data
# Replace '-' with 'N' for clarity
tech_cols = ['IR', 'LU', 'VI', 'UL', 'MC', 'ZD', 'GM']  # Adjust based on your actual columns

for col in tech_cols:
    df_rules[col] = df_rules[col].replace('-', 'N')

# Check the result
print("\nCleaned rules:")
print(df_rules.head())





Cleaned rules:
                      Component Type IR LU VI UL MC ZD GM
0                   Electric Motors   S  N  P  S  P  N  N
1     Rolling Element Bearings (Oil)  N  P  P  S  N  N  N
2  Rolling Element Bearings (Grease)  N  N  P  P  N  N  N
3         Gearboxes / Speed Reducers  S  P  N  S  N  N  S
4                        Belt Drives  N  N  P  N  N  N  N


In [3]:

# Cell 3: Create lookup function
def get_tech_needs(component_type, rules_df, include_secondary=True):
    """
    Given a component type, return dict of technology needs
    
    Parameters:
    - component_type: str, name of component
    - rules_df: dataframe with applicability rules
    - include_secondary: bool, whether to include P-rated techs as needs
    
    Returns: dict like {'IR': 'S', 'VI': 'P', 'LU': 'N', ...}
    """
    tech_cols = ['IR', 'LU', 'VI', 'UL', 'MC', 'ZD', 'GM']
    
    # Find the row for this component
    component_row = rules_df[rules_df['Component Type'] == component_type]
    
    # If not found, return all N
    if component_row.empty:
        return {tech: 'N' for tech in tech_cols}
    
    # Extract tech columns and convert to dict
    tech_needs = component_row[tech_cols].iloc[0].to_dict()
    
    # If include_secondary is False, convert P to N
    if not include_secondary:
        tech_needs = {k: ('N' if v == 'P' else v) for k, v in tech_needs.items()}
    
    return tech_needs



In [4]:
# Cell 4: Test it
test_cases = [
    'Centrifugal Pumps',
    'Electric Motors (AC Induction)',
    'Rolling Element Bearings (Grease)'
]

for component in test_cases:
    needs = get_tech_needs(component, df_rules)
    print(f"\n{component}:")
    print(needs)





Centrifugal Pumps:
{'IR': 'S', 'LU': 'P', 'VI': 'P', 'UL': 'S', 'MC': 'S', 'ZD': 'N', 'GM': 'N'}

Electric Motors (AC Induction):
{'IR': 'N', 'LU': 'N', 'VI': 'N', 'UL': 'N', 'MC': 'N', 'ZD': 'N', 'GM': 'N'}

Rolling Element Bearings (Grease):
{'IR': 'N', 'LU': 'N', 'VI': 'P', 'UL': 'P', 'MC': 'N', 'ZD': 'N', 'GM': 'N'}


In [5]:
# Cell 5: Create a helper to check if tech is needed
def is_tech_needed(component_type, technology, rules_df, primary_only=False):
    """
    Check if a specific technology is needed for a component
    
    Returns: True if needed (S or P rating), False otherwise
    If primary_only=True, only returns True for S ratings
    """
    pass

In [6]:
# Cell 6: Save for later
df_rules.to_pickle('component_map.pkl')